In [1]:
import findspark
findspark.init()

import re
from pyspark.sql.functions import col, udf, explode, when, year, month
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, LongType, StringType, DateType, ArrayType, BooleanType

from pyspark.sql import SparkSession
import pyspark as ps
config = ps.SparkConf().setAll([
    ('spark.network.timeout', '3601s'),
    ('spark.executor.heartbeatInterval', '3600s'),
])
sc = ps.SparkContext('local[*]', '', conf=config)
spark = SparkSession(sc)

In [23]:
import pyspark.pandas as ps
import datetime
import seaborn as sns

In [4]:
#!head -n 2 $PATH_METADATA_SRC
!PowerShell.exe gc "$PATH_METADATA_SRC" -head 2

gc : L'acc�s au chemin d'acc�s 'C:\Projet_MA1\Ada\project_Youniverse\ada-2022-project-fraisetagada\ada-2022-project-fra
isetagada\data\generated\yt_metadata_sub_urls.parquet' est refus�.
Au caract�re Ligne:1 : 1
+ gc ../data/generated/yt_metadata_sub_urls.parquet -head 2
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : PermissionDenied: (C:\Projet_MA1\A...ub_urls.parquet:String) [Get-Content], Unauthorized 
   AccessException
    + FullyQualifiedErrorId : GetContentReaderUnauthorizedAccessError,Microsoft.PowerShell.Commands.GetContentCommand
 


In [27]:
FILE = "../data/generated/yt_test.parquet"
schema = StructType([
    StructField("categories",    StringType(),  True),
    StructField("channel_id",    StringType(),  True),
    StructField("crawl_date",    DateType(),    True),
    StructField("description",   StringType(),  True),
    StructField("dislike_count", DoubleType(),  True), # This field must be specified as a double as it is represented as a floating point number
    StructField("display_id",    StringType(),  True),
    StructField("duration",      IntegerType(), True),
    StructField("like_count",    DoubleType(),  True), # This field must be specified as a double as it is represented as a floating point number
    StructField("tags",          StringType(),  True),
    StructField("title",         StringType(),  True),
    StructField("upload_date",   DateType(),    True),
    StructField("view_count",    DoubleType(),  True)  # This field must be specified as a double as it is represented as a floating point number
])

metadatas_df = spark.read.parquet(FILE, schema=schema)

# Cast the dislike_count, like_count and view_count to their respective integer type
metadatas_df = metadatas_df \
    .withColumn("dislike_count", metadatas_df.dislike_count.cast(IntegerType())) \
    .withColumn("like_count", metadatas_df.like_count.cast(IntegerType())) \
    .withColumn("view_count", metadatas_df.view_count.cast(LongType()))


In [32]:
# We convert the pyspark dataframe into a pyspark.pandas dataframe
ps_metadatas_df = metadatas_df.pandas_api(index_col="display_id")

In [33]:
def diff_month(d1,d2):
    
    return (d1.year - d2.year) * 12 + d1.month - d2.month



ps_metadatas_df['month_cnt'] = ps_metadatas_df['upload_date'].apply(diff_month, d2=ps_metadatas_df['upload_date'].min())

In [34]:
ps_metadatas_df.head()

,categories,channel_id,crawl_date,dislike_count,duration,like_count,tags,title,upload_date,view_count,urls,urls_count,has_urls,month_cnt
display_id,,,,,,,,,,,,,,
PCWIxrl-bcI,Music,UCzWLloEwMWZYnUErRF5WWCg,2019-11-19,17,63,507,"piano,pianist,cover,live,musician,creative,tut...",Gravity Falls Theme (Piano Tutorial),2018-01-27,32051,"[https://mnot.es/DarioDaversa, https://www.dar...",7,true,135
P-hm8-F4jGw,News & Politics,UCzSSoloGEz10HALUAbYhngQ,2019-11-13,0,64,5,"ski,crash,snowboarding,fail,epicfail,snowboard...",Snowboarder hit by chairlift,2015-04-01,828,"[https://www.newsflare.com/video/19498..., htt...",5,true,102
CYUBJanMRH4,Entertainment,UCzR7770PbrKcG9OYGzBep9w,2019-10-30,5,301,221,"raees,raees trailer,shah rukh khan,Raees Trail...",Shah Rukh Khan Contributed IMMENSELY To Raees ...,2017-01-10,12182,"[http://bit.ly/1uV6Qba, http://www.bollywoodhu...",2,true,123
mcjGa3d8SGk,Entertainment,UCzR7770PbrKcG9OYGzBep9w,2019-10-30,4,294,27,"saiyami kher,mirzya,harshvardhan kapoor,mirzya...",How Well Do You Know Rakyesh Mehra Quiz With H...,2016-09-28,7264,"[http://bit.ly/1uV6Qba, http://www.bollywoodhu...",2,true,119
QQyWK46rYII,Entertainment,UCzR7770PbrKcG9OYGzBep9w,2019-10-30,0,609,24,"bollywood hungama,bollywood hungama interviews...",Mischa Barton exclusive FULL Interview On Bhop...,2014-10-07,6036,"[http://bit.ly/1uV6Qba, http://www.bollywoodhu...",2,true,96
